In [72]:
import pandas as pd
import numpy as np
import n2n4m.preprocessing as preprocessing

In [73]:
df = pd.read_json("sample_data_expanded.json")

In [74]:
df_cleaned = preprocessing.drop_bad_bands(df)

In [75]:
df_cleaned

,Coordinates,Pixel_Class,Image_Name,1.021,1.02755,1.0341,1.04065,1.0472,1.05375,1.0603,...,3.41706,3.42371,3.43036,3.43701,3.44366,3.45031,3.45696,3.46361,3.47026,3.47692
0,"[103, 133]",[38],0BABA,0.388404,0.392957,0.392129,0.392495,0.394316,0.395171,0.393743,...,0.215394,0.217245,0.212999,0.208958,0.211876,0.215470,0.218346,0.223316,0.230014,0.235478
1,"[619, 322]",[11],13EEF,0.240750,0.236074,0.234877,0.234915,0.232637,0.232832,0.233831,...,0.191475,0.196177,0.200335,0.203132,0.205053,0.207474,0.211143,0.213895,0.217235,0.221842
2,"[466, 320]",[19],1212A,0.298133,0.291013,0.292459,0.292706,0.292531,0.290050,0.289595,...,0.198606,0.203143,0.207512,0.209775,0.212203,0.216155,0.220975,0.225380,0.226431,0.227022
3,"[310, 415]",[18],0A425,0.213243,0.209865,0.211122,0.210992,0.209650,0.210886,0.210754,...,0.116573,0.117995,0.120967,0.124153,0.125312,0.126289,0.128732,0.131439,0.132947,0.135774
4,"[623, 353]",[6],0C467,0.224506,0.220508,0.222128,0.220713,0.217933,0.218168,0.218398,...,0.094127,0.096445,0.098330,0.100216,0.102650,0.104481,0.105986,0.109464,0.112507,0.116213


In [288]:
dummy_df = df_cleaned.copy(deep=True)
dummy_df["Image_Name"] = "0BABA"
dummy_df.iloc[:1, 1] = 38
dummy_df.iloc[1:, 1] = 1

In [289]:
dummy_df

,Coordinates,Pixel_Class,Image_Name,1.021,1.02755,1.0341,1.04065,1.0472,1.05375,1.0603,...,3.41706,3.42371,3.43036,3.43701,3.44366,3.45031,3.45696,3.46361,3.47026,3.47692
0,"[103, 133]",38,0BABA,0.388404,0.392957,0.392129,0.392495,0.394316,0.395171,0.393743,...,0.215394,0.217245,0.212999,0.208958,0.211876,0.215470,0.218346,0.223316,0.230014,0.235478
1,"[619, 322]",1,0BABA,0.240750,0.236074,0.234877,0.234915,0.232637,0.232832,0.233831,...,0.191475,0.196177,0.200335,0.203132,0.205053,0.207474,0.211143,0.213895,0.217235,0.221842
2,"[466, 320]",1,0BABA,0.298133,0.291013,0.292459,0.292706,0.292531,0.290050,0.289595,...,0.198606,0.203143,0.207512,0.209775,0.212203,0.216155,0.220975,0.225380,0.226431,0.227022
3,"[310, 415]",1,0BABA,0.213243,0.209865,0.211122,0.210992,0.209650,0.210886,0.210754,...,0.116573,0.117995,0.120967,0.124153,0.125312,0.126289,0.128732,0.131439,0.132947,0.135774
4,"[623, 353]",1,0BABA,0.224506,0.220508,0.222128,0.220713,0.217933,0.218168,0.218398,...,0.094127,0.096445,0.098330,0.100216,0.102650,0.104481,0.105986,0.109464,0.112507,0.116213


Things to change:  
- Remove any .iloc
- Split up if possible

In [290]:
label_cols = ["Image_Name", "Pixel_Class", "Coordinates"]

In [291]:
def label_list_to_string(dataset:pd.DataFrame)->pd.DataFrame:
    dataset["Pixel_Class"] = dataset["Pixel_Class"].apply(lambda x: x[0])
    return dataset

def label_string_to_list(dataset:pd.DataFrame)->pd.DataFrame:
    dataset["Pixel_Class"] = dataset["Pixel_Class"].apply(lambda x: [x])
    return dataset

In [292]:
def impute_mean(dataset:pd.DataFrame, threshold=1)->pd.DataFrame:
    data = dataset.drop(columns=label_cols)
    label_data = dataset[label_cols]
    data[data>threshold] = np.nan
    data = data.fillna(data.mean())
    dataset = pd.concat([label_data, data], axis=1)
    return dataset

In [293]:
def impute_bad_values_v2(dataset:pd.DataFrame)->pd.DataFrame:    
    dataset = label_list_to_string(dataset)
    for image_name in dataset["Image_Name"].unique():
        for pixel_class in dataset[dataset["Image_Name"] == image_name][
            "Pixel_Class"
        ].unique():
            class_subset = dataset[
                (dataset["Image_Name"] == image_name)
                & (dataset["Pixel_Class"] == pixel_class)
            ]
            
            class_subset = impute_mean(class_subset, threshold=1)

            dataset[
                (dataset["Image_Name"] == image_name)
                & (dataset["Pixel_Class"] == pixel_class)
            ] = class_subset
        
        dataset[dataset["Image_Name"] == image_name] = impute_mean(
            dataset[dataset["Image_Name"] == image_name], threshold=1
        )

    
            
    dataset = label_string_to_list(dataset)
    return dataset

In [294]:
dummy_df = label_string_to_list(dummy_df)

In [295]:
test_df_2 = impute_bad_values_v2(dummy_df)

In [296]:
test_df_2.isna().sum().sum()

3

In [297]:
test_df_2

,Coordinates,Pixel_Class,Image_Name,1.021,1.02755,1.0341,1.04065,1.0472,1.05375,1.0603,...,3.41706,3.42371,3.43036,3.43701,3.44366,3.45031,3.45696,3.46361,3.47026,3.47692
0,0BABA,[38],"[103, 133]",0.388404,0.392957,0.392129,0.392495,0.394316,0.395171,0.393743,...,0.215394,0.217245,0.212999,0.208958,0.211876,0.215470,0.218346,0.223316,0.230014,0.235478
1,0BABA,[1],"[619, 322]",0.240750,0.236074,0.234877,0.234915,0.232637,0.232832,0.233831,...,0.191475,0.196177,0.200335,0.203132,0.205053,0.207474,0.211143,0.213895,0.217235,0.221842
2,0BABA,[1],"[466, 320]",0.298133,0.291013,0.292459,0.292706,0.292531,0.290050,0.289595,...,0.198606,0.203143,0.207512,0.209775,0.212203,0.216155,0.220975,0.225380,0.226431,0.227022
3,0BABA,[1],"[310, 415]",0.213243,0.209865,0.211122,0.210992,0.209650,0.210886,0.210754,...,0.116573,0.117995,0.120967,0.124153,0.125312,0.126289,0.128732,0.131439,0.132947,0.135774
4,0BABA,[1],"[623, 353]",0.224506,0.220508,0.222128,0.220713,0.217933,0.218168,0.218398,...,0.094127,0.096445,0.098330,0.100216,0.102650,0.104481,0.105986,0.109464,0.112507,0.116213


In [264]:
test_df_2_data = test_df_2.drop(columns=label_cols)
test_df_2_data[test_df_2_data>1] = np.nan
test_df_2_data.isna().sum().sum()

0

In [231]:
test_df_2

,1.021,1.02755,1.0341,1.04065,1.0472,1.05375,1.0603,1.06685,1.07341,1.07996,...,3.43701,3.44366,3.45031,3.45696,3.46361,3.47026,3.47692,Image_Name,Pixel_Class,Coordinates
1,0.240750,0.236074,0.234877,0.234915,0.232637,0.232832,0.233831,0.233048,0.232728,0.232845,...,0.203132,0.205053,0.207474,0.211143,0.213895,0.217235,0.221842,0BABA,1.0,"[619, 322]"
2,0.298133,0.291013,0.292459,0.292706,0.292531,0.290050,0.289595,0.289792,0.290770,0.287347,...,0.209775,0.212203,0.216155,0.220975,0.225380,0.226431,0.227022,0BABA,1.0,"[466, 320]"
3,0.213243,0.209865,0.211122,0.210992,0.209650,0.210886,0.210754,0.209258,0.210198,0.209418,...,0.124153,0.125312,0.126289,0.128732,0.131439,0.132947,0.135774,0BABA,1.0,"[310, 415]"
4,0.224506,0.220508,0.222128,0.220713,0.217933,0.218168,0.218398,0.217584,0.217716,0.216249,...,0.100216,0.102650,0.104481,0.105986,0.109464,0.112507,0.116213,0BABA,1.0,"[623, 353]"


In [135]:
dummy_df_num = dummy_df.drop(columns=label_cols)
dummy_df_num[dummy_df_num>1] = np.nan

In [140]:
dummy_df_num.iloc[4].isna().sum()

0

In [131]:
dummy_df

,Coordinates,Pixel_Class,Image_Name,1.021,1.02755,1.0341,1.04065,1.0472,1.05375,1.0603,...,3.41706,3.42371,3.43036,3.43701,3.44366,3.45031,3.45696,3.46361,3.47026,3.47692
0,"[103, 133]",38,0BABA,0.388404,0.392957,0.392129,0.392495,0.394316,0.395171,0.393743,...,0.215394,0.217245,0.212999,0.208958,0.211876,0.215470,0.218346,0.223316,0.230014,0.235478
1,"[619, 322]",38,0BABA,0.240750,0.236074,0.234877,0.234915,0.232637,0.232832,0.233831,...,0.191475,0.196177,0.200335,0.203132,0.205053,0.207474,0.211143,0.213895,0.217235,0.221842
2,"[466, 320]",1,0BABA,0.298133,0.291013,0.292459,0.292706,0.292531,0.290050,0.289595,...,0.198606,0.203143,0.207512,0.209775,0.212203,0.216155,0.220975,0.225380,0.226431,0.227022
3,"[310, 415]",1,0BABA,0.213243,0.209865,0.211122,0.210992,0.209650,0.210886,0.210754,...,0.116573,0.117995,0.120967,0.124153,0.125312,0.126289,0.128732,0.131439,0.132947,0.135774
4,"[623, 353]",1,0BABA,0.224506,0.220508,0.222128,0.220713,0.217933,0.218168,0.218398,...,0.094127,0.096445,0.098330,0.100216,0.102650,0.104481,0.105986,0.109464,0.112507,0.116213
